# 可信度加權決策賦予新聞可信度

## 說明

本文乃XXX的實作，請參考該文。

TODO 另寫一篇文章，來講解這裡的東西，用小論文的格式，放在readme

### 變數定義
變數名稱|中文|說明
:----|:----:|----
reporter|報導者|
news|新聞|
reader|讀者|
review|評分|對新聞打分數
reviewer|評分者|對新聞打分數的讀者
judge|評價|對評分打分數
judger|評價者|對評分打分數的讀者
score|分數|即為可信度，同weight
weight|權重|即為可信度，同score

### 測試資料

#### 初始化

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%run src/helper.py

num = {
    
    # 報導者人數
    'reporter' : 8,
    
    # 每個報導者生產新聞數
    'news_per_reporter' : 40,
    
    # 讀者人數
    'reader' : 20,
    
    # 讀者舊權重保留筆數
    'reader_pass_weight_window' : 10 
}

#### 分數範圍管理

In [ ]:
class Scale:
    def __init__(self, min = 1, max = 10):
        self.min = min
        self.max = max
        self.mean = (min + max) / 2
        self.sigma = (self.max - self.mean) / 3
        
    def arange(self):
        return np.arange(self.min, self.max + 1)

    def translateZ(self, z_score):
        translated = z_score * self.sigma + self.mean
        return max(min(translated, self.max), self.min)

#### 讀者

In [ ]:
def get_readers(n):
    readers = pd.DataFrame({
        'reader_id' : np.arange(n)
    })
    return readers

get_readers(num['reader']).head()

#### 讀者權重管理

In [ ]:
class ReaderWeightHolder:
    
    # 目前的權重
    weights = {}
    
    # 過去的權重
    past_weights = {}

    def __init__(self, readers, init_mean, reader_pass_weight_window = 10):
        self.readers = readers
        self.reader_pass_weight_window = reader_pass_weight_window
        self.init_mean = init_mean

        self.reset()

    def reset(self):
        self.past_weights = {reader_id: np.repeat(self.init_mean, self.reader_pass_weight_window).tolist() for reader_id in self.readers['reader_id']}
        self.weights = {reader_id: self.init_mean for reader_id in self.readers['reader_id']}

    def get(self, reader_id):
        return self.weights[reader_id]

    def inserts(self, new_weights):
        
        for reader_id, new_weight in new_weights.items():  
            self.past_weights[reader_id].pop(0)
            self.past_weights[reader_id].append(new_weight)
            # TODO 簡單平均數、加權平均數、做對事情，權重越高
            self.weights[reader_id] = sum(self.past_weights[reader_id]) / self.reader_pass_weight_window

    def print_weights(self, n = 5):
        return pd.DataFrame.from_dict(self.weights, orient = 'index', columns = ['weight']).rename_axis('user_id').head(n)
    
    def print_past_weights(self, n = 5):
        return pd.DataFrame.from_dict(self.past_weights, orient = 'index').rename_axis('user_id').head(n)

    def print_all(self):
        print(self.weights)
        print(self.past_weights)

##### 讀者現有權重

In [ ]:
ReaderWeightHolder(get_readers(num['reader']), Scale().mean, num['reader_pass_weight_window']).print_weights()

##### 讀者過去權重

In [ ]:
ReaderWeightHolder(get_readers(num['reader']), Scale().mean, num['reader_pass_weight_window']).print_past_weights()

#### 報導者

In [ ]:
def get_reporters(n, scale):
    
    # 常態分佈下的分數機率
    z_scores = np.random.randn(n)
    
    # 測試不公正的讀者
    z_scores[0] = -2.5
    
    # 換算成實際分數
    scores = [scale.translateZ(z_score) for z_score in z_scores]
    
    reporters = pd.DataFrame({
        'reporter_score': scores,
        'reporter_z_scores': z_scores
    })

    return index_as_id(reporters, 'reporter')

get_reporters(num['reader'], Scale()).head()

#### 新聞

In [ ]:
def get_news(per_reporter, scale, reporters):
        
    # 報導者column
    reporter_ids = list(reporters['reporter_id']) * per_reporter
    
    #新聞報導者的可信度z分數
    reporter_z_scores = list(reporters['reporter_z_scores']) * per_reporter
    
    # 定義每則新聞的可信度z分數，因為與報導者相關，將其定義為 N(author.z_score, 1)，所以後面要再加上報導者的可信度
    news_z_scores = np.random.randn(len(reporter_ids)) + reporter_z_scores
    
    # 新聞的真實可信度
    news_scores = [scale.translateZ(news_z_score) for news_z_score in news_z_scores]
    
    # TODO 視覺化一下，是否可信度高的報導者會擁有可信度高的新聞

    news = pd.DataFrame({
        'reporter_id' : reporter_ids,
        'news_score' : news_scores, 
        'news_z_score': news_z_scores
    })
    
    return index_as_id(news, 'news')

def test_get_news():
    scale = Scale()
    reporters = get_reporters(num['reporter'], scale)
    
    return get_news(num['news_per_reporter'], scale, reporters)

test_get_news().head()

#### 評分結果

In [ ]:
def get_reviews(scale, reviewers, news):
    
    # 先準備每個評分者要評分的每則新聞
    prepared = [[reviewer_id, n.news_id, n.reporter_id, n.news_score, n.news_z_score] for reviewer_id in reviewers['reader_id'] for n in news.itertuples(index = False)]
    reviews = pd.DataFrame(prepared, columns = ['reviewer_id', 'news_id', 'reporter_id', 'news_score', 'news_z_score'])

    # TODO 可以與讀者的可信度做連結?
    # TODO 視覺化一下，是否可信度高的讀者會做出可信度高的評分
    
    # 每個讀者對每則新聞的評分
    
    #如果是均勻分布的話，且公正評分與公正評價，那似乎會以7.25為平均線
    #reviews['review_score'] = np.random.choice(scale.arange(), len(prepared))
    
    # 評分與新聞的可信度相關
    reviews['review_z_score'] = np.random.randn(len(reviews)) + reviews['news_z_score']
    reviews['review_score'] = [scale.translateZ(review_z_score) for review_z_score in reviews['review_z_score']]

    return index_as_id(reviews, 'review')

def test_get_reviews():
    scale = Scale()
    readers = get_readers(num['reader'])
    reporters = get_reporters(num['reporter'] , scale)
    news = get_news(num['news_per_reporter'], scale, reporters)
    
    return get_reviews(scale, readers, news)

test_get_reviews().head()

#### 評價結果

In [ ]:
def get_judges(scale, reviewers, reviews, news):
    
    # 每個評價者要評價的每則評分
    pairs = [[review.review_id, review.review_score, review.news_score, review.reporter_id, judger_id] for review in reviews.itertuples(index = False) for judger_id in reviewers['reader_id']]
    judges = pd.DataFrame(pairs, columns = ['review_id', 'review_score', 'news_score', 'reporter_id', 'judger_id'])

    # 評價公式，評分的分數與實際的分數越準確，評價分數越高(假設所有讀者都是理性的)
    judging = lambda review_score, real_score: scale.max - abs(review_score - real_score)
    
    # 計算每則評分的評價分數
    judges['judge_score'] = [judging(judge.review_score, judge.news_score) for judge in judges.itertuples(index = False)]
    
    return index_as_id(judges, 'judge')

def test_get_judges():
    scale = Scale()
    readers = get_readers(num['reader'])
    reporters = get_reporters(num['reporter'] , scale)
    news = get_news(num['news_per_reporter'], scale, reporters)
    reviews = get_reviews(scale, readers, news)
    
    return get_judges(scale, readers, reviews, news)

test_get_judges().head()

#### 合併在一起，方便後續測試

In [ ]:
def get_data(news, reviews, judges, reader_weight_holder):
    cols_to_use = lambda right, left: left.columns.difference(right.columns)

    data = judges.merge(reviews[cols_to_use(judges, reviews)], left_on = 'review_id', right_index = True)
    data = data.merge(news[cols_to_use(data, news)], left_on = 'news_id', right_index = True)
    data['judger_weight'] = [reader_weight_holder.get(judger_id) for judger_id in data['judger_id']]

    # 排除自己評自己的
    data = data[data.judger_id != data.reviewer_id]

    return data

def test_get_data():
    scale = Scale()
    readers = get_readers(num['reader'])
    reporters = get_reporters(num['reporter'] , scale)
    news = get_news(num['news_per_reporter'], scale, reporters)
    reviews = get_reviews(scale, readers, news)
    judges = get_judges(scale, readers, reviews, news)
    reader_weight_holder = ReaderWeightHolder(readers, scale.mean)
    
    return get_data(news, reviews, judges, reader_weight_holder)
    
test_get_data().head()

## 計算可信度 / 權重

### 每個讀者的可信度/權重

In [ ]:
def get_reviewer_weights(data):
    # 我想知道，這個judge的評價加權後是多少，所以分數要乘以某個權重，權重是倍數的概念，來自於之前做了多少對的事
    # 這個評價分數代表評分的品質，評分的品質，也代表評分者的品質，因為每個judge_score代表其他讀者對自己的評價

    # 讀者的權重= (其他讀者對這個讀者的評分的評價 * 其他讀者的權重) 的 加權平均
    # weight(user) = sum(score(judge_u1)*weight(user1) + score(judge_u2)*weight(user2) + ... + score(judge_uN)*weight(userN)) / sum(weights)

    judger_weight_averaging = lambda x: np.average(x['judge_score'], weights = x.judger_weight)
    reviewer_weights = data.groupby('reviewer_id').apply(judger_weight_averaging)
    
    return reviewer_weights

get_reviewer_weights(test_get_data()).head()

### 更新讀者權重

In [ ]:
def update_reader_weight(data, reader_weight_holder):
    data['reviewer_weight'] = [reader_weight_holder.get(reviewer_id) for reviewer_id in data['reviewer_id']]
    return data

### 可信度/權重計算公式

In [ ]:
review_weight_averaging = lambda x: np.average(x['review_score'], weights = x.reviewer_weight)
review_averaging = lambda x:np.average(x['review_score'])

def get_score(data, key, averaging_func = review_weight_averaging):
    return data.groupby(key).apply(averaging_func)

# 計算新聞的可信度/權重
def get_news_score(data, averaging_func = review_weight_averaging):
    return get_score(data, 'news_id', averaging_func)

# 計算報導者的可信度/權重
def get_reporter_score(data, averaging_func = review_weight_averaging):
    return get_score(data, 'reporter_id', averaging_func)

# 視覺化報導者可信度
def draw_reporter_scores(data, reporters):
    
    plt.subplot(1, 1, 1)
    plt.xticks(np.arange(0, len(reporters), 1.0))

    # 評分直接計算
    reporter_scores = get_reporter_score(data, review_averaging)
    plt.plot(reporter_scores, label = 'simple avg')
    
    # 評分*評分者可信度計算
    reporter_scores_weighted = get_reporter_score(data)
    plt.plot(reporter_scores_weighted, label = 'weighted avg')
    
    # 報導者可信度
    plt.plot(reporters['reporter_score'], 'k--', label = 'real', c = '0.55')
    
    plt.legend(loc = 'upper right')
    plt.show()

def draw_user_weights(weights, scale):
    weights = list(weights)
    plt.xticks(np.arange(0, len(weights), 1.0))
    plt.plot(weights)

### 公正的讀者

In [ ]:
def simulate(times, scale, readers, reporters):
    
    reader_weight_holder = ReaderWeightHolder(readers, scale.mean)
    
    for i in range(times):
        news = get_news(num['news_per_reporter'], scale, reporters)
        reviews = get_reviews(scale, readers, news)
        judges = get_judges(scale, readers, reviews, news)
        data = get_data(news, reviews, judges, reader_weight_holder)
        
        reviewer_weights = get_reviewer_weights(data)
        reader_weight_holder.inserts(reviewer_weights.to_dict())
        update_reader_weight(data, reader_weight_holder)

    draw_reporter_scores(data, reporters)
    draw_user_weights(reader_weight_holder.weights.values(), scale)

def run_simulate(times):
    
    scale = Scale()
    
    properties = {
        'times' : times,
        'scale' : scale,
        'readers' : get_readers(num['reader']),
        'reporters' : get_reporters(num['reporter'] , scale)
    }
    
    simulate(**properties)

run_simulate(10)

### 不公正的讀者

In [ ]:
# 特定讀者
def get_unfair_readers(readers, percent = 0.2):
    unfair_readers = readers.sample(frac = percent).sort_values(by = 'reader_id')
    print('%d out of %d unfair readers with frac %.2f:' % (len(unfair_readers), len(readers), percent), list(unfair_readers['reader_id']))
    return unfair_readers

# 特定讀者會給出的分數
def get_unfair_scores():
    return [9, 10]

# 特定報導者
def get_target_reporters(reporters, percent = 0.2):
    return reporters.iloc[0:1]

#### 評分不公正，評價公正

In [ ]:
# 什麼叫做不公正，就是遇到特定報導者寫出來的新聞，會一律評高分或低分
# TODO 這邊寫出有0% 20% 40% 50% 60%的不公正評分者情況下，實際的報導者可信度會是多少
def fill_unfair_reviews(reviews, readers, reporters, unfair_readers, unfair_scores, target_reporters):

    grep_unfair_reviews = reviews['reviewer_id'].isin(unfair_readers['reader_id'])
    grep_unfair_reviews &= reviews['reporter_id'].isin(target_reporters['reporter_id'])
    
    # 符合以上條件的評分
    unfair_reviews = reviews[grep_unfair_reviews]
    
    # 修改評分分數
    reviews.loc[grep_unfair_reviews, 'review_score'] = np.random.choice(unfair_scores, len(unfair_reviews))
    
    return reviews

In [ ]:
def simulate_unfair_review(times, scale, unfair_scores, readers, unfair_readers, reporters, target_reporters):
    
    reader_weight_holder = ReaderWeightHolder(readers, scale.mean)
        
    for i in range(times):
        news = get_news(num['news_per_reporter'], scale, reporters)
        reviews = get_reviews(scale, readers, news)
        reviews = fill_unfair_reviews(reviews, readers, reporters, unfair_readers, unfair_scores, target_reporters)
        judges = get_judges(scale, readers, reviews, news)
        data = get_data(news, reviews, judges, reader_weight_holder)
        
        reviewer_weights = get_reviewer_weights(data)
        reader_weight_holder.inserts(reviewer_weights.to_dict())
        update_reader_weight(data, reader_weight_holder)

    draw_reporter_scores(data, reporters)
    draw_user_weights(reader_weight_holder.weights.values(), scale)

def run_simulate_unfair_review(times, unfair_reader_percent = 0.4):
    
    scale = Scale()
    readers = get_readers(num['reader'])
    reporters = get_reporters(num['reporter'] , scale)
    
    properties = {
        'times' : times,
        'scale' : scale,
        'unfair_scores' : get_unfair_scores(),
        'readers' : readers,
        'unfair_readers' : get_unfair_readers(readers, unfair_reader_percent),
        'reporters' : reporters,
        'target_reporters' : get_target_reporters(reporters)
    }

    simulate_unfair_review(**properties)
    
run_simulate_unfair_review(10)

#### 評分公正，評價不公正

In [ ]:
# 這邊跳過，直接探討下一個好了
# 什麼叫做不公正，就是遇到特定報導者寫出來的新聞的評分，如果太低的話會評低分（打壓），高的話則是高分（讚賞）
# 這邊寫出有0% 20% 40% 50% 60%的不公正評價者情況下，實際的報導者可信度會是多少
def fill_unfair_judges(judges, readers, reporters, unfair_readers, target_reporters):

    grep_unfair_judges = judges['judger_id'].isin(unfair_readers['reader_id'])
    grep_unfair_judges &= judges['reporter_id'].isin(target_reporters['reporter_id'])
    
    # 符合以上條件的評分
    unfair_judges = judges[grep_unfair_judges]
    
    # 修改評分分數
    judges.loc[grep_unfair_judges, 'judge_score'] = judges['review_score']
    
    return judges

In [ ]:
def simulate_unfair_judges(times, scale, readers, unfair_readers, reporters, target_reporters):
    
    reader_weight_holder = ReaderWeightHolder(readers, scale.mean)
    
    for i in range(times):
        news = get_news(num['news_per_reporter'], scale, reporters)
        reviews = get_reviews(scale, readers, news)
        judges = get_judges(scale, readers, reviews, news)
        judges = fill_unfair_judges(judges, readers, reporters, unfair_readers, target_reporters)
        data = get_data(news, reviews, judges, reader_weight_holder)
        
        reviewer_weights = get_reviewer_weights(data)
        reader_weight_holder.inserts(reviewer_weights.to_dict())
        update_reader_weight(data, reader_weight_holder)

    draw_reporter_scores(data, reporters)
    draw_user_weights(reader_weight_holder.weights.values(), scale)

def run_simulate_unfair_judges(times, unfair_reader_percent = 0.4):
    scale = Scale()
    readers = get_readers(num['reader'])
    reporters = get_reporters(num['reporter'] , scale)
    
    properties = {
        'times' : times,
        'scale' : scale,
        'readers' : readers,
        'unfair_readers' : get_unfair_readers(readers, unfair_reader_percent),
        'reporters' : reporters,
        'target_reporters' : get_target_reporters(reporters)
    }
    
    simulate_unfair_judges(**properties)

run_simulate_unfair_judges(10)

#### 評分和評價都不公正

In [ ]:
def simulate_unfair_review_and_judge(times, scale, unfair_scores, readers, unfair_readers, reporters, target_reporters, drawing = True):
    
    reader_weight_holder = ReaderWeightHolder(readers, scale.mean)
    
    for i in range(times):
        news = get_news(num['news_per_reporter'], scale, reporters)
        reviews = get_reviews(scale, readers, news)
        reviews = fill_unfair_reviews(reviews, readers, reporters, unfair_readers, unfair_scores, target_reporters)
        judges = get_judges(scale, readers, reviews, news)
        judges = fill_unfair_judges(judges, readers, reporters, unfair_readers, target_reporters)
        data = get_data(news, reviews, judges, reader_weight_holder)

        reviewer_weights = get_reviewer_weights(data)
        reader_weight_holder.inserts(reviewer_weights.to_dict())
        update_reader_weight(data, reader_weight_holder)

    if drawing:
        draw_reporter_scores(data, reporters)
        draw_user_weights(reader_weight_holder.weights.values(), scale)
    
    return data, reporters

def run_simulate_unfair_review_and_judge(times, unfair_reader_percent = 0.4):
    scale = Scale()
    readers = get_readers(num['reader'])
    reporters = get_reporters(num['reporter'] , scale)
    
    properties = {
        'times' : times,
        'scale' : scale,
        'readers' : readers,
        'unfair_scores' : get_unfair_scores(),
        'unfair_readers' : get_unfair_readers(readers, unfair_reader_percent),
        'reporters' : reporters,
        'target_reporters' : get_target_reporters(reporters)
    }
    
    dummy1, dummy2 = simulate_unfair_review_and_judge(**properties)
    
run_simulate_unfair_review_and_judge(10)

##### 極度不公正的讀者 

In [ ]:
# 會發現simple avg和weighted avg差不了太多，而且都與real差太遠，有可能就是因為其他公正的評分評價拉高了不公正讀者的權重
# 因此這邊測試極度不公正的讀者，或者也可以說是專門帶風向的讀者
def remove_fair_reviews(reviews, unfair_readers, target_reporters):
    
    # 不公正讀者
    grep_fair_reviews = reviews['reviewer_id'].isin(unfair_readers['reader_id'])
    
    # 對其他報導者的公正評分
    grep_fair_reviews &= ~reviews['reporter_id'].isin(target_reporters['reporter_id'])

    return reviews.drop(reviews[grep_fair_reviews].index)

def remove_fair_judges(judges, unfair_readers, target_reporters):
    
    # 不公正讀者
    grep_fair_judges = judges['judger_id'].isin(unfair_readers['reader_id'])
    
    # 對其他評分的公正評價
    grep_fair_judges &= ~judges['reporter_id'].isin(target_reporters['reporter_id'])
    
    return judges.drop(judges[grep_fair_judges].index)

In [ ]:
def simulate_extremely_unfair_review_and_judge(times, scale, unfair_scores, readers, unfair_readers, reporters, target_reporters, drawing = True):
    
    reader_weight_holder = ReaderWeightHolder(readers, scale.mean)
    
    for i in range(times):
        news = get_news(num['news_per_reporter'], scale, reporters)
        
        reviews = get_reviews(scale, readers, news)
        reviews = fill_unfair_reviews(reviews, readers, reporters, unfair_readers, unfair_scores, target_reporters)
        reviews = remove_fair_reviews(reviews, unfair_readers, target_reporters)
        
        judges = get_judges(scale, readers, reviews, news)
        judges = fill_unfair_judges(judges, readers, reporters, unfair_readers, target_reporters)
        judges = remove_fair_judges(judges, unfair_readers, target_reporters)
        
        data = get_data(news, reviews, judges, reader_weight_holder)

        reviewer_weights = get_reviewer_weights(data)
        reader_weight_holder.inserts(reviewer_weights.to_dict())
        update_reader_weight(data, reader_weight_holder)

    if drawing:
        draw_reporter_scores(data, reporters)
        draw_user_weights(reader_weight_holder.weights.values(), scale)
    
    return data, reporters

def run_simulate_extremely_unfair_review_and_judge(times, unfair_reader_percent = 0.4):
    scale = Scale()
    readers = get_readers(num['reader'])
    reporters = get_reporters(num['reporter'] , scale)
    
    properties = {
        'times' : times,
        'scale' : scale,
        'readers' : readers,
        'unfair_scores' : get_unfair_scores(),
        'unfair_readers' : get_unfair_readers(readers, unfair_reader_percent),
        'reporters' : reporters,
        'target_reporters' : get_target_reporters(reporters)
    }
    
    dummy1, dummy2 = simulate_extremely_unfair_review_and_judge(**properties)
    
run_simulate_extremely_unfair_review_and_judge(10)

### 不公正讀者比例對可信度的影響

In [ ]:
def simulate_unfair_reader_percents(times, unfair_reader_percents):

    scale = Scale()
    readers = get_readers(num['reader'])
    reporters = get_reporters(num['reporter'], scale)
    results = pd.DataFrame(columns = ['percent', 'simple_avg', 'weight_avg'])

    for percent in unfair_reader_percents:

        properties = {
            'times' : times,
            'scale' : scale,
            'readers' : readers,
            'unfair_scores' : get_unfair_scores(),
            'unfair_readers' : get_unfair_readers(readers, percent),
            'reporters' : reporters,
            'target_reporters' : get_target_reporters(reporters),
            'drawing' : False
        }

        data, reporters = simulate_extremely_unfair_review_and_judge(**properties)
        data = data[data.reporter_id == 0]

        # 評分直接計算
        reporter_scores = get_reporter_score(data, review_averaging)

        # 評分*評分者可信度計算
        reporter_scores_weighted = get_reporter_score(data)
        
        result = {
            'percent': percent,
            'simple_avg': reporter_scores[0],
            'weight_avg': reporter_scores_weighted[0],
            'reporter_score': reporters.iloc[0].reporter_score
        }
        results = results.append(result, ignore_index = True)
        
    return results

def run_simulate_unfair_reader_percents():
    
    unfair_reader_percents = [0.0, 0.1, 0.2, 0.4, 0.6, 0.8]
    results = simulate_unfair_reader_percents(10, unfair_reader_percents)
    results.set_index('percent', inplace = True)

    plt.plot(results['simple_avg'], label = 'simple avg')
    plt.plot(results['weight_avg'], label = 'weighted avg')
    plt.plot(results['reporter_score'], 'k--', label = 'real', c = '0.55')
    plt.legend(loc = 'upper left')
    plt.show()

run_simulate_unfair_reader_percents()

#### 次數增加是否能讓加權平均後的可信度接近真實可信度

In [ ]:
def simulate_unfair_reader_times(times, unfair_reader_percent = 0.4):
    
    scale = Scale()
    readers = get_readers(num['reader'])
    reporters = get_reporters(num['reporter'], scale)
    results = pd.DataFrame(columns = ['time', 'simple_avg', 'weight_avg'])
    
    for time in times:
        
        properties = {
            'times' : time,
            'scale' : scale,
            'readers' : readers,
            'unfair_scores' : get_unfair_scores(),
            'unfair_readers' : get_unfair_readers(readers, unfair_reader_percent),
            'reporters' : reporters,
            'target_reporters' : get_target_reporters(reporters),
            'drawing' : False
        }
        
        data, reporters = simulate_extremely_unfair_review_and_judge(**properties)
        data = data[data.reporter_id == 0]

        # 評分直接計算
        reporter_scores = get_reporter_score(data, review_averaging)

        # 評分*評分者可信度計算
        reporter_scores_weighted = get_reporter_score(data)
                
        result = {
            'time': time,
            'simple_avg': reporter_scores[0],
            'weight_avg': reporter_scores_weighted[0],
            'reporter_score': reporters.iloc[0].reporter_score
        }
        
        results = results.append(result, ignore_index = True)
        
    return results
    
def run_simulate_unfair_reader_times():
    
    times = [5, 10, 20, 30, 50, 70, 90]
    results = simulate_unfair_reader_times(times)
    results.set_index('time', inplace = True)

    plt.plot(results['simple_avg'], label = 'simple avg')
    plt.plot(results['weight_avg'], label = 'weighted avg')
    plt.plot(results['reporter_score'], 'k--', label = 'real', c = '0.55')
    plt.legend(loc = 'upper left')
    plt.show()
    
run_simulate_unfair_reader_times()